In [240]:
# import shutil

# file_name = "open.zip"
# # output_dir = "dataset/dataset"
# output_dir = "."

# format = "zip"
# shutil.unpack_archive(file_name, output_dir, format)

In [241]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
# import random

SEED = 777

# random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
# torch.backends.cudnn.deterministic = True

In [242]:
df_train = pd.read_csv('train.csv')
print(df_train.head(3))

        img_id                 img_path  airplane  airport  bare soil  \
0  TRAIN_00000  ./train/TRAIN_00000.jpg         0        0          1   
1  TRAIN_00001  ./train/TRAIN_00001.jpg         0        0          1   
2  TRAIN_00002  ./train/TRAIN_00002.jpg         0        0          1   

   baseball diamond  basketball court  beach  bridge  buildings  ...  tanks  \
0                 0                 0      0       0          0  ...      0   
1                 0                 0      0       0          1  ...      0   
2                 0                 0      0       0          0  ...      0   

   tennis court  terrace  track  trail  transmission tower  trees  water  \
0             0        0      0      0                   0      1      0   
1             0        0      0      0                   0      0      0   
2             0        0      0      0                   0      1      0   

   wetland  wind turbine  
0        0             0  
1        0             0  
2   

In [243]:
df_test = pd.read_csv('test.csv')
print(df_test.head(3))

       img_id               img_path
0  TEST_00000  ./test/TEST_00000.jpg
1  TEST_00001  ./test/TEST_00001.jpg
2  TEST_00002  ./test/TEST_00002.jpg


In [244]:
print(len(df_train), len(df_test)) # 65496, 43665

65496 43665


In [245]:
# df_train.info()

In [246]:
# _ = df_train.hist(figsize=(20, 20))

In [247]:
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms # , models

# 이미지 경로와 레이블을 나눕니다
image_paths = df_train['img_path'].values
labels = df_train.iloc[:, 2:].values  # 레이블 열 선택

In [248]:
X_train, X_val, y_train, y_val = train_test_split(image_paths, labels, test_size=0.2, random_state=42)

print(X_train)
print(X_val)
print(y_train)
print(y_val)

['./train/TRAIN_33119.jpg' './train/TRAIN_10054.jpg'
 './train/TRAIN_53906.jpg' ... './train/TRAIN_00860.jpg'
 './train/TRAIN_15795.jpg' './train/TRAIN_56422.jpg']
['./train/TRAIN_31596.jpg' './train/TRAIN_18550.jpg'
 './train/TRAIN_36293.jpg' ... './train/TRAIN_39893.jpg'
 './train/TRAIN_26601.jpg' './train/TRAIN_33248.jpg']
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 1 ... 1 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 1 ... 0 0 0]]
[[0 0 1 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]]


In [249]:
# 사용자 정의 데이터셋 클래스
class SatelliteDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx].astype(np.float32)

        if self.transform:
            image = self.transform(image)

        return image, label

In [250]:
# def __init__(..., labels=None, mode='train'):

# def __getitem__(self, idx):
#     if self.mode == 'train':
#       x = self.data[idx]
#       y = self.labels[idx]
#       return x, y
#     else:
#       x = self.data[idx]
#       return x

# Dataset(..., mode='train')

In [251]:
# 이미지 전처리 및 데이터 증강
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Larger size for more complex models
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomApply([
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1)
    ], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [252]:
train_dataset = SatelliteDataset(X_train, y_train, transform=train_transform)
val_dataset = SatelliteDataset(X_val, y_val, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

In [253]:
from torchvision.models import resnet50

# ResNet50 모델을 불러오고, 최상위 레이어를 제거합니다
model = resnet50(pretrained=True)

# 분류 레이어를 변경합니다
num_classes = labels.shape[1]
print(num_classes)

model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(512, num_classes),
    nn.Sigmoid()
)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


60


In [254]:
# class MultiLabelResNet(nn.Module):
#     def __init__(self, num_classes):
#         super(MultiLabelResNet, self).__init__()
#         self.model = models.resnet50(pretrained=True)
#         self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

#     def forward(self, x):
#         return torch.sigmoid(self.model(x))

In [255]:
# # 사전 학습된 레이어들은 학습되지 않도록 동결합니다
# for layer in base_model.layers:
#     layer.trainable = False

In [256]:
from torch.optim import lr_scheduler

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

model = model.to(device)

criterion = nn.BCELoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=len(train_loader), epochs=50)

cuda


In [257]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for images, labels in train_loader:
            # print(images.shape) # [32, 3, 224, 224]
            # print(labels.shape) # [32, 60]
            images = images.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # 배치의 평균 손실 값에 배치 크기를 곱하여 현재 배치의 총 손실 값을 계산
            #   배치 크기가 달라질 경우를 대비하여 전체 데이터셋에 대한 정확한 평균 손실 값을 계산하기 위함
            train_loss += loss.item() * images.size(0)
        
        train_loss /= len(train_loader.dataset)

        if epoch >= 9:
            torch.save(model.state_dict(), f'epoch{epoch}.pt')
        
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device)
                
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * images.size(0)
        
        val_loss /= len(val_loader.dataset)
        
        scheduler.step()
        
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

In [258]:
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=50) # scheduler , epochs=50)

Epoch 1/50, Train Loss: 0.1633, Val Loss: 0.1063


KeyboardInterrupt: 

In [265]:
model.load_state_dict(torch.load('epoch38.pt'))

<All keys matched successfully>

In [266]:
# 모델 평가 함수
def evaluate_model(model, val_loader):
    model.eval()
    val_loss = 0.0
    corrects = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item() * images.size(0)
            
            preds = outputs > 0.5
            # print(preds.shape) # [32, 60]
            # corrects: 올바르게 예측된 레이블의 총 수
            # print(preds == labels)
            # ? 데이콘이 각 데이터의 모든 레이블이 맞아야 맞다고 체점하는지
            corrects += (preds == labels).sum().item()
    
    val_loss /= len(val_loader.dataset)
    # 전체 검증 데이터셋의 모든 레이블에 대한 정확도
    accuracy = corrects / (len(val_loader.dataset) * labels.size(1))
    
    print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {accuracy:.4f}')

In [267]:
evaluate_model(model, val_loader)

Validation Loss: 0.0520, Validation Accuracy: 0.9791


In [268]:
class TestDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, img_path

In [269]:
test_dataset = TestDataset(df_test['img_path'].values, transform=val_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

In [270]:
model.eval()

predictions = []

with torch.no_grad():
    for images, img_paths in test_loader:
        images = images.to(device)
        outputs = model(images)
        # outputs = outputs > 0.5
        outputs = outputs.cpu().numpy()
        
        for img_path, output in zip(img_paths, outputs):
            img_id = img_path.split('/')[-1].split('.')[0]  # 파일 이름에서 img_id 추출
            # predictions.append([img_id] + output.tolist())
            predictions.append([img_id] + [col for col in output.tolist()])
            # print(predictions)

In [271]:
np.array(predictions).shape

(43665, 61)

In [272]:
# # 새로운 이미지에 대한 예측 함수
# def predict_image(model, image_path):
#     model.eval()
#     image = Image.open(image_path).convert('RGB')
#     # transform = transforms.Compose([
#     #     transforms.Resize((224, 224)),
#     #     transforms.ToTensor()
#     # ])
#     # image = transform(image).unsqueeze(0).to(device)
    
#     image = val_transform(image).unsqueeze(0).to(device)
    
#     with torch.no_grad():
#         output = model(image)
#     # ?? 제출 양식 왜 확률이지
    
#     return output.cpu().numpy()

In [273]:
# predictions = []
# for img_id, img_path in zip(df_test['img_id'], df_test['img_path']):
#     pred = predict_image(model, img_path)
#     predictions.append([img_id] + pred.tolist())

In [274]:
sample_submission = pd.read_csv('sample_submission.csv')
print(sample_submission.head(3))

       img_id  airplane  airport  bare soil  baseball diamond  \
0  TEST_00000         0        0          0                 0   
1  TEST_00001         0        0          0                 0   
2  TEST_00002         0        0          0                 0   

   basketball court  beach  bridge  buildings  cars  ...  tanks  tennis court  \
0                 0      0       0          0     0  ...      0             0   
1                 0      0       0          0     0  ...      0             0   
2                 0      0       0          0     0  ...      0             0   

   terrace  track  trail  transmission tower  trees  water  wetland  \
0        0      0      0                   0      0      0        0   
1        0      0      0                   0      0      0        0   
2        0      0      0                   0      0      0        0   

   wind turbine  
0             0  
1             0  
2             0  

[3 rows x 61 columns]


In [275]:
columns = [col for col in df_train.columns if col != 'img_path']
df_predictions = pd.DataFrame(predictions, columns=columns)

In [276]:
class_columns = [col for col in df_train.columns if col not in ['img_id', 'img_path']]
# df_predictions[class_columns] = df_predictions[class_columns].astype(int)
# df_predictions[class_columns] = df_predictions[class_columns].round(2)
df_predictions

,img_id,airplane,airport,bare soil,baseball diamond,basketball court,beach,bridge,buildings,cars,...,tanks,tennis court,terrace,track,trail,transmission tower,trees,water,wetland,wind turbine
0,TEST_00000,7.122533e-17,1.252369e-15,0.000013,1.377356e-26,3.027507e-13,5.802833e-12,2.288399e-18,0.000231,9.502489e-09,...,4.368716e-13,1.307474e-11,5.247762e-09,1.958257e-14,3.864470e-07,1.603567e-08,0.002248,0.000011,7.547775e-09,8.717879e-20
1,TEST_00001,2.042777e-10,1.642777e-10,0.861272,3.610178e-21,1.523815e-11,3.414202e-14,5.685124e-17,1.000000,9.998770e-01,...,2.435036e-11,3.603454e-11,1.638018e-10,1.539852e-11,1.611498e-09,1.255012e-10,0.999877,0.005178,4.689848e-12,1.003897e-26
2,TEST_00002,4.444573e-07,6.065267e-10,0.678852,3.712249e-15,1.368452e-09,2.161503e-10,6.784780e-07,0.049860,9.284833e-01,...,4.811736e-08,6.522838e-07,2.212666e-07,1.565127e-08,2.063420e-05,6.402930e-07,0.301665,0.000005,1.140984e-08,1.851786e-14
3,TEST_00003,2.090319e-05,5.493608e-05,0.627948,2.127815e-09,3.009371e-05,4.805267e-05,1.049549e-02,0.180833,4.284928e-01,...,1.232054e-03,1.310741e-03,8.574809e-03,1.100413e-03,1.093662e-02,4.654031e-04,0.991535,0.051700,1.238674e-04,4.009687e-09
4,TEST_00004,2.869463e-07,2.575255e-07,0.229427,3.598879e-07,9.993396e-01,2.019457e-09,5.498687e-08,0.759775,5.472131e-01,...,3.209471e-07,1.903473e-06,6.965385e-07,2.022771e-03,1.044015e-04,3.732358e-07,0.490771,0.000064,8.786441e-08,9.284461e-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43660,TEST_43660,5.156089e-12,3.324739e-11,0.000034,1.145958e-17,3.801710e-06,2.568055e-13,6.674786e-10,0.999947,9.971169e-01,...,2.497656e-07,1.278369e-05,1.070977e-07,1.341101e-06,6.215746e-06,8.092380e-08,0.902448,0.035549,1.080540e-08,9.642080e-21
43661,TEST_43661,1.480062e-08,7.016538e-09,0.003190,5.976247e-10,9.999002e-01,1.171631e-09,1.482069e-10,0.141470,1.942284e-02,...,2.723258e-08,1.322393e-06,1.438852e-07,1.974784e-06,2.228372e-05,2.187507e-07,0.356037,0.000006,5.602574e-08,2.636658e-16
43662,TEST_43662,5.235693e-04,3.095625e-04,0.739654,5.993159e-04,8.916872e-05,1.595895e-05,2.271070e-05,0.417297,1.112014e-01,...,4.879758e-05,1.803340e-04,1.058874e-03,2.326487e-02,4.522066e-01,6.478430e-05,0.658082,0.153801,5.228743e-04,3.575902e-08
43663,TEST_43663,1.806959e-05,3.588037e-04,0.908139,2.998814e-07,7.276814e-07,1.428136e-07,1.997791e-03,0.558091,4.642875e-02,...,3.779907e-07,6.446212e-08,5.320634e-07,2.754557e-05,1.205022e-01,3.233155e-06,0.008879,0.289396,1.688543e-03,9.951506e-01


In [277]:
df_predictions.to_csv('submission.csv', index=False)